In [ ]:
import pandas as pd

path = '/home/django/source/bhp056_project/bhp056/'
fname = path + 'identifiers.csv'
f = open(fname, 'r')
lines = f.readlines()

identifiers = []
count = 0
lines.pop(0)
for line in lines:
    line = line.strip()
    line = line.split(',')
    identifiers.append(line[1])
maternal_locators = MaternalLocator.objects.filter(maternal_visit__appointment__registered_subject__subject_identifier__in=identifiers)
data = []
for maternal_locator in maternal_locators:
    d = maternal_locator.__dict__
    mail_address = d.get('mail_address')
    mail_address = mail_address.replace('\r\n', ' ')
    d.update(mail_address=mail_address)
    physical_address = d.get('physical_address')
    physical_address = physical_address.replace('\r\n', ' ')
    d.update(physical_address=physical_address)
    contact_physical_address = d.get('contact_physical_address')
    contact_physical_address = contact_physical_address.replace('\r\n', ' ')
    d.update(contact_physical_address=contact_physical_address)
    subject_work_place = d.get('subject_work_place')
    subject_work_place = subject_work_place.replace('\r\n', ' ')
    d.update(subject_work_place=subject_work_place)
    d.update(subject_identifier=maternal_locator.maternal_visit.appointment.registered_subject.subject_identifier)
    data.append(d)
df = pd.DataFrame(data)
df.to_csv('maternal_locator.csv', sep='|', encoding='utf-8')

In [ ]:
import pandas as pd
from django.core.exceptions import MultipleObjectsReturned
path = '/home/django/source/bhp056_project/bhp056/'
fname = path + 'identifiers.csv'
f = open(fname, 'r')
lines = f.readlines()

identifiers = []
count = 0
lines.pop(0)
found = 0
missing_l = []
miss = 0
duplicate = 0
duplicated = []
for line in lines:
    line = line.strip()
    line = line.split(',')
    identifiers.append(line[1])
    try:
        MaternalLocator.objects.get(maternal_visit__appointment__registered_subject__subject_identifier=line[1])
    except MaternalLocator.DoesNotExist:
        missing_l.append(line[1])
        miss += 1
    except MultipleObjectsReturned:
        duplicated.append(line[1])
        duplicate += 1
    else:
        found += 1
    count += 1
print('Missing locators:', miss)
print('Total locators', count)
print('Found locators', found)
print('Duplicate locators', duplicate)

In [ ]:
maternal_data = []
infant_data = []
for maternal_locator in maternal_locators:
    dm = {}
    di = {}
    dm.update(subject_identifier=maternal_locator.maternal_visit.appointment.registered_subject.subject_identifier)
    consent = MaternalConsent.objects.filter(subject_identifier=maternal_locator.maternal_visit.appointment.registered_subject.subject_identifier).last()
    dm.update(first_name=consent.first_name)
    dm.update(last_name=consent.last_name)
    infants = InfantBirth.objects.filter(registered_subject__relative_identifier=maternal_locator.maternal_visit.appointment.registered_subject.subject_identifier)
    for infant in infants:
        di.update(first_name=infant.first_name, subject_identifier=infant.registered_subject.subject_identifier)
    maternal_data.append(dm)
    infant_data.append(di)
df1 = pd.DataFrame(maternal_data)
df1.to_csv('maternal_names.csv', sep='|', encoding='utf-8')
df2 = pd.DataFrame(infant_data)
df2.to_csv('child_names.csv', sep='|', encoding='utf-8')